In [1]:
import os
from utils import get_vocabs

In [2]:
data_dir = 'data_old'
# path_train = os.path.join(data_dir, 'train1_short.wtag')
# path_test = os.path.join(data_dir, 'test1_short.wtag')
path_train = os.path.join(data_dir, 'train.wtag')
path_test = os.path.join(data_dir, 'test.wtag')
paths_list = [path_train, path_test]
word_dict, pos_dict = get_vocabs(paths_list)

In [3]:
from data_handling import PosDataset
from torch.utils.data.dataloader import DataLoader
print("path_train -", path_train)
print("path_test -", path_test)
BATCH_SIZE = 1
paths_list = [path_train, path_test]
word_dict, pos_dict = get_vocabs(paths_list)
train = PosDataset(word_dict, pos_dict, data_dir, 'train', padding=False)
train_dataloader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
test = PosDataset(word_dict, pos_dict, data_dir, 'test', padding=False)
test_dataloader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

path_train - data_old/train.wtag
path_test - data_old/test.wtag
idx_pos_mappings - [0, 1, '#', '$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
pos_idx_mappings - {'<pad>': 0, '<unk>': 1, '#': 2, '$': 3, "''": 4, ',': 5, '-LRB-': 6, '-RRB-': 7, '.': 8, ':': 9, 'CC': 10, 'CD': 11, 'DT': 12, 'EX': 13, 'FW': 14, 'IN': 15, 'JJ': 16, 'JJR': 17, 'JJS': 18, 'MD': 19, 'NN': 20, 'NNP': 21, 'NNPS': 22, 'NNS': 23, 'PDT': 24, 'POS': 25, 'PRP': 26, 'PRP$': 27, 'RB': 28, 'RBR': 29, 'RBS': 30, 'RP': 31, 'SYM': 32, 'TO': 33, 'UH': 34, 'VB': 35, 'VBD': 36, 'VBG': 37, 'VBN': 38, 'VBP': 39, 'VBZ': 40, 'WDT': 41, 'WP': 42, 'WP$': 43, 'WRB': 44, '``': 45}
idx_pos_mappings - [0, 1, '#', '$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 

In [4]:
print("Number of Train Tagged Sentences ", len(train))
print("Number of Test Tagged Sentences ",len(test))

Number of Train Tagged Sentences  5000
Number of Test Tagged Sentences  1000


In [5]:
import torch
from torch import nn
import torch.optim as optim
from model import DnnPosTagger
from trainer import Trainer

WORD_EMBEDDING_DIM = 100
HIDDEN_DIM = 100
NUM_LAYERS = 2
word_vocab_size = len(train.word_idx_mappings)
tag_vocab_size = len(train.pos_idx_mappings)

model = DnnPosTagger(train_dataloader.dataset.word_vectors, HIDDEN_DIM, NUM_LAYERS, word_vocab_size, tag_vocab_size)

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

if use_cuda:
    model.cuda()
    
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
acumulate_grad_steps = 50 # This is the actual batch_size, while we officially use batch_size=1
trainer = Trainer(model, optimizer, loss_function, device)

word embeddings shape: torch.Size([15212, 300])


In [6]:
EPOCHS = 15
acumulate_grad_steps = 50
len_train = len(train)
len_test = len(test)
trainer.train(EPOCHS, train_dataloader, test_dataloader, acumulate_grad_steps, len_train, len_test, early_stopping=5)

/home/student/hw2/code_dir/trainer.py:103: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  acc += torch.mean(torch.tensor(pos_idx_tensor.to("cpu") == indices.to("cpu"), dtype=torch.float))


Epoch: 1 | Loss: 0.0137 | Training accuracy: 97.065% | Test accuracy: 94.506% | Epoch Time: 36.19 secs
saving model
Epoch: 2 | Loss: 0.0020 | Training accuracy: 98.391% | Test accuracy: 94.846% | Epoch Time: 34.54 secs
saving model
Epoch: 3 | Loss: 0.0011 | Training accuracy: 98.910% | Test accuracy: 94.753% | Epoch Time: 34.66 secs
Epoch: 4 | Loss: 0.0007 | Training accuracy: 99.161% | Test accuracy: 94.789% | Epoch Time: 35.31 secs
Epoch: 5 | Loss: 0.0005 | Training accuracy: 99.519% | Test accuracy: 95.005% | Epoch Time: 34.94 secs
saving model
Epoch: 6 | Loss: 0.0003 | Training accuracy: 99.691% | Test accuracy: 94.868% | Epoch Time: 35.53 secs
Epoch: 7 | Loss: 0.0002 | Training accuracy: 99.802% | Test accuracy: 94.991% | Epoch Time: 34.59 secs
Epoch: 8 | Loss: 0.0002 | Training accuracy: 99.803% | Test accuracy: 94.757% | Epoch Time: 34.80 secs
Epoch: 9 | Loss: 0.0001 | Training accuracy: 99.858% | Test accuracy: 94.775% | Epoch Time: 36.02 secs
Epoch: 10 | Loss: 0.0001 | Trainin